### How accurate is analyst recommendation for stocks?

And if they are somewhat accurate, what is a typical time horizon for the momentum to halt?

In [1]:
%matplotlib inline

# auto reload modules in dev
%load_ext autoreload
%autoreload 2

import yfinance as yf
import pandas as pd

import sys
print(sys.path)

['/Users/zhehaowang/projects/zhehao.me/notebooks', '/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/usr/local/Cellar/python/3.7.4_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/Users/zhehaowang/Library/Python/3.7/lib/python/site-packages', '/usr/local/lib/python3.7/site-packages', '/Users/zhehaowang/projects/yfinance', '/Users/zhehaowang/Library/Python/3.7/lib/python/site-packages/IPython/extensions', '/Users/zhehaowang/.ipython']


In [3]:
ticker = yf.Ticker("FB")
ticker.info['recommendationMean']

1.9

In [4]:
# unique actions for each firm
# ticker.recommendations.groupby("Firm")["Action"].apply(set)

In [5]:
# Unique To Grade for each firm
# ticker.recommendations.groupby("Firm")["To Grade"].apply(set)

In [6]:
# ticker.calendar

In [7]:
# get latest analyst ratings
import dateutil

date_cutoff = dateutil.parser.parse("2020-03-01")
# ticker.recommendations.tail()

latest_rec = ticker.recommendations.sort_values('Date').groupby("Firm").tail(1)
filtered = latest_rec[latest_rec.index > date_cutoff]
print(filtered)

                                           Firm           To Grade From Grade  \
Date                                                                            
2020-03-17 15:27:04                     Wedbush         Outperform              
2020-03-18 16:39:53                   JP Morgan         Overweight              
2020-03-26 15:48:08                   Citigroup                Buy              
2020-04-29 14:47:06                 BMO Capital     Market Perform              
2020-04-30 10:15:08            Pivotal Research               Hold       Sell   
2020-04-30 11:05:26                       Baird         Outperform              
2020-04-30 11:40:01                  Rosenblatt                Buy              
2020-04-30 12:04:35              JMP Securities  Market Outperform              
2020-04-30 12:08:52               Raymond James         Strong Buy              
2020-04-30 12:14:20                MKM Partners                Buy              
2020-04-30 12:25:31         

In [145]:
rdf = ticker.recommendations
# verifying they indeed don't have later actions
rdf[rdf['Firm'] == 'Argus Research']

,Firm,To Grade,From Grade,Action
Date,,,,
2012-05-21 07:51:00,Argus Research,Buy,,up
2020-03-27 13:03:50,Argus Research,Buy,Hold,up


In [8]:
# this needs to be fixed in src lib
ticker.financials

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [9]:
# this needs to be fixed in src lib
ticker.quarterly_financials

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [10]:
ticker.major_holders

,0,1
0,0.66%,% of Shares Held by All Insider
1,79.41%,% of Shares Held by Institutions
2,79.94%,% of Float Held by Institutions
3,3459,Number of Institutions Holding Shares


In [11]:
ticker.institutional_holders

,Holder,Shares,Date Reported,% Out,Value
0,"Vanguard Group, Inc. (The)",188828724,2020-03-30,0.0785,31496631163
1,Blackrock Inc.,159221271,2020-03-30,0.0662,26558108002
2,"FMR, LLC",124247137,2020-03-30,0.0517,20724422451
3,Price (T.Rowe) Associates Inc,108819909,2020-03-30,0.0453,18151160821
4,State Street Corporation,95449004,2020-03-30,0.0397,15920893867
5,Capital Research Global Investors,72328001,2020-03-30,0.0301,12064310566
6,Capital International Investors,51583155,2020-03-30,0.0215,8604070254
7,Capital World Investors,40433503,2020-03-30,0.0168,6744308300
8,"Geode Capital Management, LLC",34812327,2020-03-30,0.0145,5806696143
9,Northern Trust Corporation,27734123,2020-03-30,0.0115,4626051716


In [12]:
# this needs to be fixed in src lib
ticker.balance_sheet

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [13]:
# this needs to be fixed in src lib
ticker.cashflow

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [14]:
# this needs to be fixed in src lib
ticker.earnings

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [15]:
# this needs to be fixed in src lib
ticker.quarterly_earnings

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [16]:
ticker.actions

,Dividends,Stock Splits
Date,,


In [18]:
sp500df = pd.read_csv("sp500.csv")
sp500df.head()

,Symbol,Security,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


### What are some highest rated stocks?

In [40]:
# build a giant stocks DF, write them to data file for analysis for later on
import os

symbols = list(sp500df['Symbol'])
# print(symbols.index('BF.B'))
res_dict = {}
dest_dir = '~/projects/yfinance/data/'
for s in symbols:
    # print(s)
    t = yf.Ticker(s)
    res_dict[s] = t.info
    recs = t.recommendations
    if recs is not None:
        recs.to_csv(os.path.join(dest_dir, '{}_ratings.csv'.format(s)))

MMM
ABT
ABBV
ABMD
ACN
ATVI
ADBE
AMD
AAP
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
ADS
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
ANTM
AON
AOS
APA
AIV
AAPL
AMAT
APTV
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
BKR
BLL
BAC
BK
BAX
BDX
BRK.B
BBY
BIIB
BLK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BF.B
CHRW
COG
CDNS
CPB
COF
CAH
KMX
CCL
CARR
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
COO
CPRT
GLW
CTVA
COST
COTY
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DRE
DD
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EOG
EFX
EQIX
EQR
ESS
EL
EVRG
ES
RE
EXC
EXPE
EXPD
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FE
FRC
FISV
FLT
FLIR
FLS
FMC
F
FTNT
FTV
FBHS
FOXA
FOX
BEN
FCX
GPS
GRMN
IT
GD
GE
GIS
GM
GPC
GILD
GL
GPN
GS
GWW
HRB
HAL
HBI
HOG
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
HPE
HLT
HFC
HOLX
HD
HON
HRL
HST
HWM
HPQ


In [41]:
info_df = pd.DataFrame.from_dict(res_dict).transpose()
info_df.to_csv(os.path.join(dest_dir, 'sp500_info.csv'))

In [44]:
info_df.sort_values('recommendationMean').head(10)

,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,state,country,companyOfficers,website,...,totalRevenue,totalCashPerShare,financialCurrency,revenuePerShare,quickRatio,recommendationMean,regularMarketPrice,logo_url,fax,address2
J,75201,Industrials,48000,Jacobs Engineering Group Inc. provides consult...,Dallas,214-583-8500,TX,United States,[],http://www.jacobs.com,...,13349712896,14.118,USD,99.476,1.911,1.6,82.83,https://logo.clearbit.com/jacobs.com,NaN,Suite 1200
LKQ,60661,Consumer Cyclical,51000,"LKQ Corporation, together with its subsidiarie...",Chicago,312 621 1950,IL,United States,[],http://www.lkqcorp.com,...,12406740992,1.095,USD,40.288,0.774,1.6,27.56,https://logo.clearbit.com/lkqcorp.com,312 621 1969,Suite 2800
PWR,77056,Industrials,39500,"Quanta Services, Inc. provides specialty contr...",Houston,713-629-7600,TX,United States,[],http://www.quantaservices.com,...,12068988928,2.74,USD,82.922,1.673,1.6,38.31,https://logo.clearbit.com/quantaservices.com,NaN,Suite 2600
BSX,01752-1234,Healthcare,36000,"Boston Scientific Corporation develops, manufa...",Marlborough,508-683-4000,MA,United States,[],http://www.bostonscientific.com,...,10785000448,0.264,USD,7.737,0.521,1.6,35.95,https://logo.clearbit.com/bostonscientific.com,NaN,NaN
GOOG,94043,Communication Services,123048,Alphabet Inc. provides online advertising serv...,Mountain View,650-253-0000,CA,United States,[],http://www.abc.xyz,...,166677004288,171.734,USD,241.388,3.508,1.6,1428.49,https://logo.clearbit.com/abc.xyz,NaN,NaN
KMX,23238,Consumer Cyclical,27050,"CarMax, Inc., through its subsidiaries, operat...",Richmond,804 747 0422,VA,United States,[],http://www.carmax.com,...,21424087040,0.358,USD,129.972,0.162,1.6,91,https://logo.clearbit.com/carmax.com,NaN,NaN
DXCM,92121,Healthcare,3900,"DexCom, Inc., a medical device company, focuse...",San Diego,858 200 0200,CA,United States,[],http://www.dexcom.com,...,1600600064,16.357,USD,17.498,5.284,1.6,368.599,https://logo.clearbit.com/dexcom.com,NaN,NaN
AIZ,10005,Financial Services,14000,"Assurant, Inc., through its subsidiaries, prov...",New York,212-859-7000,NY,United States,[],http://www.assurant.com,...,10052599808,37.833,USD,163.603,0.173,1.7,109.25,https://logo.clearbit.com/assurant.com,NaN,41st Floor
IQV,27703,Healthcare,67000,IQVIA Holdings Inc. provides advanced analytic...,Durham,919 998 2000,NC,United States,[],http://www.iqvia.com,...,11157999616,5.184,USD,57.59,0.94,1.7,142.59,https://logo.clearbit.com/iqvia.com,NaN,NaN
LHX,32919,Industrials,50000,"L3Harris Technologies, Inc., an aerospace and ...",Melbourne,321-727-9100,FL,United States,[],http://www.l3harris.com,...,21424001024,4.637,USD,87.072,0.9,1.7,201.11,https://logo.clearbit.com/l3harris.com,NaN,NaN


In [49]:
info_df.sort_values('dividendYield', ascending=False).head(10)

,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,state,country,companyOfficers,website,...,totalRevenue,totalCashPerShare,financialCurrency,revenuePerShare,quickRatio,recommendationMean,regularMarketPrice,logo_url,fax,address2
SPG,46204,Real Estate,3500,Simon is a global leader in the ownership of p...,Indianapolis,317-636-1600,IN,United States,[],http://www.simon.com,...,5664115200,12.182,USD,18.43,0.836,2.5,77.75,https://logo.clearbit.com/simon.com,NaN,NaN
OKE,74103,Energy,2882,"ONEOK, Inc., together with its subsidiaries, e...",Tulsa,918-588-7000,OK,United States,[],http://www.oneok.com,...,9521081344,1.284,USD,23.003,0.943,2.6,33.64,https://logo.clearbit.com/oneok.com,NaN,NaN
CTL,71203,Communication Services,42500,"CenturyLink, Inc., a facilities-based communic...",Monroe,318-388-9000,LA,United States,[],http://www.centurylink.com,...,22201999360,1.425,USD,20.69,0.674,3.3,10.18,https://logo.clearbit.com/centurylink.com,NaN,NaN
IRM,02110,Real Estate,25000,"Iron Mountain Incorporated (NYSE: IRM), founde...",Boston,617-535-4766,MA,United States,[],http://www.ironmountain.com,...,4277452032,0.53,USD,14.889,0.622,2.6,28,https://logo.clearbit.com/ironmountain.com,NaN,NaN
WMB,74172,Energy,4812,"The Williams Companies, Inc., together with it...",Tulsa,918-573-2000,OK,United States,[],http://co.williams.com,...,8060000256,0.33,USD,6.648,0.629,1.9,18.89,https://logo.clearbit.com/co.williams.com,NaN,Suite 4700
VTR,60654,Real Estate,516,"Ventas, Inc. (together with its subsidiaries, ...",Chicago,877-483-6827,IL,United States,[],http://www.ventasreit.com,...,3913996032,7.634,USD,10.579,2.897,3,39.01,https://logo.clearbit.com/ventasreit.com,NaN,Suite 3300
MO,23230,Consumer Defensive,7300,"Altria Group, Inc., through its subsidiaries, ...",Richmond,804 274 2200,VA,United States,[],http://www.altria.com,...,20452999168,3.022,USD,10.967,0.517,2.1,39.41,https://logo.clearbit.com/altria.com,NaN,NaN
WFC,94104,Financial Services,263000,"Wells Fargo & Company, a diversified financial...",San Francisco,866-249-3302,CA,United States,[],http://www.wellsfargo.com,...,75406000128,86.901,USD,17.612,None,3.1,28,https://logo.clearbit.com/wellsfargo.com,NaN,NaN
XOM,75039-2298,Energy,74900,Exxon Mobil Corporation explores for and produ...,Irving,972-940-6000,TX,United States,[],http://corporate.exxonmobil.com,...,249070993408,2.699,USD,58.33,0.498,3.1,48.04,https://logo.clearbit.com/corporate.exxonmobil...,972-940-6143,NaN
PRU,07102,Financial Services,51511,"Prudential Financial, Inc., through its subsid...",Newark,973-802-6000,NJ,United States,[],http://www.prudential.com,...,63082999808,112.18,USD,157.021,0.696,2.9,63.77,https://logo.clearbit.com/prudential.com,NaN,NaN


In [51]:
info_df.sort_values('forwardPE').head(10)

,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,state,country,companyOfficers,website,...,totalRevenue,totalCashPerShare,financialCurrency,revenuePerShare,quickRatio,recommendationMean,regularMarketPrice,logo_url,fax,address2
LYV,90210,Communication Services,10500,"Live Nation Entertainment, Inc. operates as a ...",Beverly Hills,310-867-7000,CA,United States,[],http://www.livenationentertainment.com,...,11185833984,15.463,USD,53.11,0.852,2.1,50,https://logo.clearbit.com/livenationentertainm...,NaN,NaN
HST,20817,Real Estate,175,"Host Hotels & Resorts, Inc. is an S&P 500 comp...",Bethesda,240-744-1000,MD,United States,[],http://www.hosthotels.com,...,5144000000,3.966,USD,7.123,12.761,2.6,12.82,https://logo.clearbit.com/hosthotels.com,NaN,Suite 1300
CCL,33178,Consumer Cyclical,150000,Carnival Corporation & Plc operates as a leisu...,Miami,305 599 2600,FL,United States,[],http://www.carnivalcorp.com,...,20941000704,1.977,USD,30.449,0.177,2.8,19.81,https://logo.clearbit.com/carnivalcorp.com,305 406 8630,NaN
HAL,77032,Energy,50000,Halliburton Company provides a range of servic...,Houston,281 871 2699,TX,United States,[],http://www.halliburton.com,...,21707999232,1.579,USD,24.774,1.234,2.7,13.17,https://logo.clearbit.com/halliburton.com,NaN,NaN
DVN,73102-5015,Energy,1800,"Devon Energy Corporation, an independent energ...",Oklahoma City,405 235 3611,OK,United States,[],http://www.devonenergy.com,...,6981000192,3.99,USD,17.981,1.365,2.1,13.03,https://logo.clearbit.com/devonenergy.com,NaN,NaN
RCL,33132-2096,Consumer Cyclical,85300,Royal Caribbean Cruises Ltd. operates as a cru...,Miami,305 539 6000,FL,United States,[],http://www.rclinvestor.com,...,10543643648,18.582,USD,50.364,0.455,2.3,60.6,https://logo.clearbit.com/rclinvestor.com,305 539 0562,NaN
NOV,77036-6565,Energy,33810,"National Oilwell Varco, Inc. designs, construc...",Houston,713 346 7500,TX,United States,[],http://www.nov.com,...,8422000128,2.872,USD,22.004,1.68,2.6,13.42,https://logo.clearbit.com/nov.com,NaN,NaN
MGM,89109,Consumer Cyclical,63000,"MGM Resorts International, through its subsidi...",Las Vegas,702-693-7120,NV,United States,[],http://www.mgmresorts.com,...,11552260096,12.198,USD,22.455,2.481,2.7,19.62,https://logo.clearbit.com/mgmresorts.com,NaN,NaN
NBL,77070,Energy,2282,"Noble Energy, Inc., an independent energy comp...",Houston,281 872 3100,TX,United States,[],http://www.nblenergy.com,...,4396000256,2.912,USD,9.192,1.119,2,10.29,https://logo.clearbit.com/nblenergy.com,NaN,NaN
HES,10036,Energy,1775,"Hess Corporation, an exploration and productio...",New York,212 997 8500,NY,United States,[],http://www.hess.com,...,6089999872,6.816,USD,20.109,1.476,2.2,51.11,https://logo.clearbit.com/hess.com,NaN,40th Floor
